In [0]:
from pyspark.sql.functions import col, current_timestamp

# Tabelas
raw_table = "default.raw_transactions"
core_table = "default.user_behavior_profiles"
analytics_table = "default.real_time_alerts"

# Caminho para o checkpoint deste stream específico diferente do checkpoint da ingestão
storage_account_name = "adlshydra"
checkpoint_path = f"abfss://raw@{storage_account_name}.dfs.core.windows.net/checkpoint_alerts"

In [0]:
# 1. Lendo a tabela de transações como um STREAM
transactions_stream_df = spark.readStream.table(raw_table)
# 2. Lendo a tabela de perfis como uma tabela ESTÁTICA (batch)
profiles_df = spark.read.table(core_table)

In [0]:
# Fazendo o join do stream de transações com a tabela de perfis
enriched_stream_df = transactions_stream_df.join(
    profiles_df,
    on="user_id",
    how="inner" 
)

In [0]:
# Sensibildiade pra detecção
N_STD_DEVS = 3.0

alerts_df = enriched_stream_df.filter(
    col("amount_brl") > (col("avg_amount") + N_STD_DEVS * col("stddev_amount"))
).withColumn("alert_timestamp", current_timestamp()) # Adicionamos um timestamp de quando o alerta foi gerado

# Escrevendo os alertas na Analytics
alerts_stream_query = alerts_df.writeStream \
    .format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", checkpoint_path) \
    .trigger(availableNow=True) \
    .toTable(analytics_table)

In [0]:
%sql
SELECT * FROM default.real_time_alerts